In [ ]:
import pandas as pd
import lightning as L
import lightning.pytorch as pl
from lightning.pytorch.callbacks import EarlyStopping
from pytorch_forecasting import TimeSeriesDataSet
from lightning.pytorch.loggers import TensorBoardLogger
from pytorch_forecasting.models.rnn import RecurrentNetwork
from pytorch_forecasting.data.encoders import EncoderNormalizer

In [ ]:
# Define model

data = pd.read_csv('dataset/preliminaryData/bss_activity_meteorological_popular-hours.csv')

max_prediction_length = 7*6
max_encoder_length = 15*6
num_workers=32

training_cutoff = data["time_idx"].max() - max_prediction_length

data = data[lambda x: x.time_idx < data["time_idx"].max() - 30*4*6] # drop celebration days
data["station"] = data["station"].astype(str)
data["month"] = data["month"].astype(str)
data["weekday"] = data["weekday"].astype(str)
data["is_weekend"] = data["is_weekend"].astype(str)
data["time_of_day"] = data["time_of_day"].astype(str)
data["season"] = data["season"].astype(str)

training = TimeSeriesDataSet(
    data[lambda x: x.time_idx < training_cutoff],
    group_ids=["station"],
    target="activity",
    time_idx="time_idx",
    min_encoder_length=max_encoder_length // 2,
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    time_varying_unknown_reals=["activity"],
    static_categoricals=["station"],
    time_varying_known_categoricals=["weekday", "is_weekend", "time_of_day", "month"],
    time_varying_known_reals=["is_public_hours"],
    target_normalizer=EncoderNormalizer(transformation="softplus"),
    lags={"activity": [6, 6*7,6*365]},
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
)

validation = TimeSeriesDataSet.from_dataset(training, data, predict=True, stop_randomization=True)
batch_size = 128
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=num_workers)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=num_workers)

pl.seed_everything(42)

In [ ]:
# Model tuning

import optuna
from optuna.integration import PyTorchLightningPruningCallback
from lightning.pytorch.callbacks import LearningRateMonitor, ModelCheckpoint
from pytorch_forecasting.metrics.point import MAE
import optuna.logging
import os
import pickle

model_path = "study_lstm"
log_dir = "study_lstm"
gradient_clip_val_range = [0.01, 1.0]
dropout_range = (0.1, 0.5)
rnn_layers = (1, 10)
hidden_size = (8, 128)
learning_rate_range = (1e-4, 1e-1)
n_trials = 50
max_epochs = 50
study = None

class PyTorchLightningPruningCallbackAdjusted(pl.Callback, PyTorchLightningPruningCallback):
    pass

def objective(trial: optuna.Trial) -> float:
    # Filenames for each trial must be made unique in order to access each checkpoint.
    checkpoint_callback = ModelCheckpoint(
        dirpath=os.path.join(model_path, "trial_{}".format(trial.number)), filename="{epoch}", monitor="val_loss"
    )

    learning_rate_callback = LearningRateMonitor()
    logger = TensorBoardLogger(log_dir, name="optuna", version=trial.number)
    gradient_clip_val = trial.suggest_loguniform("gradient_clip_val", *gradient_clip_val_range)

    trainer = pl.Trainer(
        accelerator="auto",
        max_epochs=max_epochs,
        gradient_clip_val=gradient_clip_val,
        callbacks=[
            learning_rate_callback,
            checkpoint_callback,
            PyTorchLightningPruningCallbackAdjusted(trial, monitor="val_loss"),
        ],
        logger=logger,
        limit_train_batches=30, 
        devices=1,
        enable_progress_bar=optuna.logging.INFO,
    )

    # create model
    model = RecurrentNetwork.from_dataset(
        train_dataloader.dataset,
        dropout=trial.suggest_uniform("dropout", *dropout_range),
        cell_type='LSTM',
        learning_rate=0.06,
        loss=MAE(),
        rnn_layers=trial.suggest_int("rnn_layers", *rnn_layers),
        hidden_size=trial.suggest_int("hidden_size", *hidden_size, log=True),
        optimizer="Ranger",
        log_interval=-1,
    )
    model.hparams.learning_rate = trial.suggest_loguniform("learning_rate", *learning_rate_range)

    # fit
    trainer.fit(model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)

    # report result
    return trainer.callback_metrics["val_loss"].item()


# setup optuna and run
if study is None:
    study = optuna.create_study(direction="minimize", pruner=optuna.pruners.SuccessiveHalvingPruner())
study.optimize(objective, n_trials=n_trials)

with open("study_lstm_2.pkl", "wb") as fout:
    pickle.dump(study, fout)

In [ ]:
# Model run

early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
logger = TensorBoardLogger("study")  # logging results to a tensorboard

trainer = pl.Trainer(
    max_epochs=50,
    accelerator='auto', 
    enable_model_summary=True,
    callbacks=[early_stop_callback],
    gradient_clip_val=0.02,
    logger=logger,
    limit_train_batches=30,
    enable_checkpointing=True,
)

net = RecurrentNetwork.from_dataset(
    training,
    cell_type='LSTM',
    learning_rate=0.06,
    rnn_layers=1,
    hidden_size=79,
    dropout=0.46659,
)

trainer.fit(
    net,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader)

In [ ]:
# Model evaluation

from pytorch_forecasting.metrics.point import MAE, SMAPE, RMSE

best_model_path = trainer.checkpoint_callback.best_model_path
best_lstm = RecurrentNetwork.load_from_checkpoint(best_model_path)
predictions = best_lstm.predict(val_dataloader, return_y=True, trainer_kwargs=dict(accelerator="cpu"))

print(MAE()(predictions.output, predictions.y))
print(SMAPE()(predictions.output, predictions.y))
print(RMSE()(predictions.output, predictions.y))